# Forest Sites Interactive Map Viewer

Interactive Leaflet maps for examining forest site locations defined in `forest_sites.yaml`.

**Purpose:** Verify site accuracy, examine forest structure, and confirm 3DEP lidar coverage.

## Features
- Interactive maps with satellite basemap
- Site bounding boxes shown as polygons (green)
- **3DEP lidar flight boundaries overlay (cyan)** - shows available lidar coverage
- Pop-up information for each site
- All 18 forest sites across the US

In [2]:
# Install ipyleaflet if not available
try:
    import ipyleaflet
except ImportError:
    import subprocess
    subprocess.check_call(['pip', 'install', 'ipyleaflet'])
    import ipyleaflet

import yaml
import json
import requests
from pathlib import Path
from shapely.geometry import shape, box
from ipyleaflet import Map, GeoJSON, basemaps, basemap_to_tiles, LayersControl, FullScreenControl, ScaleControl
from ipywidgets import HTML, Layout

print(f"ipyleaflet version: {ipyleaflet.__version__}")

ipyleaflet version: 0.20.0


In [3]:
# Load forest sites configuration
sites_yaml_path = Path('../forest_sites.yaml')
geojson_dir = Path('../geojson')

with open(sites_yaml_path, 'r') as f:
    config = yaml.safe_load(f)

sites = config['sites']
print(f"Loaded {len(sites)} forest sites")

# Load combined GeoJSON
with open(geojson_dir / 'all_sites.geojson', 'r') as f:
    all_sites_geojson = json.load(f)

print(f"GeoJSON features: {len(all_sites_geojson['features'])}")

# Fetch 3DEP lidar boundaries from Hobu Inc
print("\nFetching 3DEP lidar boundaries...")
lidar_url = 'https://raw.githubusercontent.com/hobuinc/usgs-lidar/master/boundaries/resources.geojson'
response = requests.get(lidar_url)
lidar_boundaries = response.json()
print(f"Loaded {len(lidar_boundaries['features'])} 3DEP lidar datasets")

# Calculate total point count
total_points = sum(f['properties'].get('count', 0) for f in lidar_boundaries['features'])
print(f"Total points available: {total_points:,.0f}")

Loaded 18 forest sites
GeoJSON features: 18

Fetching 3DEP lidar boundaries...
Loaded 2221 3DEP lidar datasets
Total points available: 73,773,529,099,863


In [4]:
# Check lidar coverage for each site
def check_lidar_coverage(site_bbox, lidar_geojson):
    """
    Check which lidar datasets intersect with a site's bounding box.
    
    Args:
        site_bbox: [west, south, east, north]
        lidar_geojson: GeoJSON FeatureCollection of lidar boundaries
        
    Returns:
        List of intersecting dataset names and total point estimate
    """
    west, south, east, north = site_bbox
    site_box = box(west, south, east, north)
    
    intersecting = []
    for feature in lidar_geojson['features']:
        try:
            lidar_geom = shape(feature['geometry'])
            if site_box.intersects(lidar_geom):
                intersecting.append({
                    'name': feature['properties'].get('name', 'Unknown'),
                    'count': feature['properties'].get('count', 0)
                })
        except Exception:
            continue
    
    return intersecting

# Check coverage for all sites
print("Checking 3DEP lidar coverage for each site...")
print("-" * 80)

site_coverage = {}
for site_id, site_data in sites.items():
    coverage = check_lidar_coverage(site_data['bbox'], lidar_boundaries)
    site_coverage[site_id] = coverage
    
    if coverage:
        total_points = sum(c['count'] for c in coverage)
        print(f"✓ {site_id:<30} {len(coverage):>2} dataset(s), ~{total_points:>15,} points")
    else:
        print(f"✗ {site_id:<30} NO LIDAR COVERAGE")

sites_with_coverage = sum(1 for c in site_coverage.values() if c)
print("-" * 80)
print(f"Sites with lidar coverage: {sites_with_coverage}/{len(sites)}")

Checking 3DEP lidar coverage for each site...
--------------------------------------------------------------------------------
✗ sequoia_giant_forest           NO LIDAR COVERAGE
✓ redwood_humboldt                1 dataset(s), ~ 61,750,406,838 points
✓ olympic_hoh                     1 dataset(s), ~ 21,439,817,649 points
✓ tongass_mendenhall              1 dataset(s), ~  2,211,557,952 points
✓ yellowstone_lodgepole           1 dataset(s), ~183,753,966,673 points
✓ rocky_mountain_subalpine        4 dataset(s), ~ 80,494,769,423 points
✓ coconino_ponderosa              1 dataset(s), ~ 55,223,690,056 points
✓ gila_mixed_conifer              1 dataset(s), ~ 78,485,794,991 points
✓ great_smoky_cove                3 dataset(s), ~143,940,037,923 points
✓ mark_twain_ozark                1 dataset(s), ~ 61,864,269,323 points
✓ white_mountain_northern         1 dataset(s), ~ 60,565,448,080 points
✓ boundary_waters                 3 dataset(s), ~656,640,746,172 points
✓ superior_old_growth         

## Overview Map - All Sites

This map shows all forest sites across the continental US with 3DEP lidar coverage overlay.

**Legend:**
- **Red polygons**: Forest site AOIs
- **Cyan polygons**: 3DEP lidar flight boundaries (toggle in layer control)

In [5]:
# Helper function to parse 3DEP collection names
import re

def parse_lidar_name(name):
    """
    Parse 3DEP lidar collection name to extract metadata.
    Names follow pattern: STATE_Location_Year or USGS_LPC_STATE_Location_Year
    Examples: AK_BrooksCamp_2012, USGS_LPC_CA_SanDiego_2014
    """
    if not name:
        return {'state': 'Unknown', 'location': 'Unknown', 'year': 'Unknown'}
    
    # Remove common prefixes
    clean_name = re.sub(r'^USGS_LPC_', '', name)
    
    # Split by underscore
    parts = clean_name.split('_')
    
    # State abbreviation mapping for common ones
    state_abbrevs = {
        'AK': 'Alaska', 'AL': 'Alabama', 'AR': 'Arkansas', 'AZ': 'Arizona',
        'CA': 'California', 'CO': 'Colorado', 'CT': 'Connecticut', 'DC': 'Washington DC',
        'DE': 'Delaware', 'FL': 'Florida', 'GA': 'Georgia', 'HI': 'Hawaii',
        'IA': 'Iowa', 'ID': 'Idaho', 'IL': 'Illinois', 'IN': 'Indiana',
        'KS': 'Kansas', 'KY': 'Kentucky', 'LA': 'Louisiana', 'MA': 'Massachusetts',
        'MD': 'Maryland', 'ME': 'Maine', 'MI': 'Michigan', 'MN': 'Minnesota',
        'MO': 'Missouri', 'MS': 'Mississippi', 'MT': 'Montana', 'NC': 'North Carolina',
        'ND': 'North Dakota', 'NE': 'Nebraska', 'NH': 'New Hampshire', 'NJ': 'New Jersey',
        'NM': 'New Mexico', 'NV': 'Nevada', 'NY': 'New York', 'OH': 'Ohio',
        'OK': 'Oklahoma', 'OR': 'Oregon', 'PA': 'Pennsylvania', 'RI': 'Rhode Island',
        'SC': 'South Carolina', 'SD': 'South Dakota', 'TN': 'Tennessee', 'TX': 'Texas',
        'UT': 'Utah', 'VA': 'Virginia', 'VT': 'Vermont', 'WA': 'Washington',
        'WI': 'Wisconsin', 'WV': 'West Virginia', 'WY': 'Wyoming', 'PR': 'Puerto Rico'
    }
    
    result = {'state': 'Unknown', 'location': 'Unknown', 'year': 'Unknown'}
    
    if len(parts) >= 1:
        # First part is usually state abbreviation
        state_abbrev = parts[0].upper()
        result['state'] = state_abbrevs.get(state_abbrev, state_abbrev)
    
    if len(parts) >= 2:
        # Middle parts are location, last part might be year
        # Check if last part is a year (4 digits)
        if parts[-1].isdigit() and len(parts[-1]) == 4:
            result['year'] = parts[-1]
            location_parts = parts[1:-1]
        else:
            location_parts = parts[1:]
        
        # Join location parts and add spaces before capitals (CamelCase)
        location = '_'.join(location_parts)
        location = re.sub(r'([a-z])([A-Z])', r'\1 \2', location)
        location = location.replace('_', ' ')
        result['location'] = location if location else 'Unknown'
    
    return result

# Create overview map centered on continental US
overview_map = Map(
    center=(39.0, -98.0),
    zoom=4,
    layout=Layout(width='100%', height='600px'),
    scroll_wheel_zoom=True
)

# Add satellite basemap
satellite_layer = basemap_to_tiles(basemaps.Esri.WorldImagery)
overview_map.add(satellite_layer)

# Add 3DEP lidar boundaries layer (cyan, semi-transparent)
lidar_layer = GeoJSON(
    data=lidar_boundaries,
    style={
        'color': '#00ffff',
        'weight': 1,
        'fillColor': '#00ffff',
        'fillOpacity': 0.15
    },
    hover_style={'fillOpacity': 0.3},
    name='3DEP Lidar Coverage'
)
overview_map.add(lidar_layer)

# Add all forest sites as GeoJSON layer (red)
sites_layer = GeoJSON(
    data=all_sites_geojson,
    style={'color': '#ff0000', 'weight': 3, 'fillColor': '#ff0000', 'fillOpacity': 0.3},
    hover_style={'fillOpacity': 0.6},
    name='Forest Sites'
)
overview_map.add(sites_layer)

# Create HTML widget for popup display
popup_html = HTML()
popup_html.layout.width = '350px'

# Popup callback for forest sites
def on_site_click(event, feature, **kwargs):
    props = feature['properties']
    forest_type = props.get('forest_type', 'N/A')
    if forest_type and forest_type != 'N/A':
        forest_type = forest_type.replace('_', ' ').title()
    popup_html.value = f"""
    <div style="font-family: Arial, sans-serif; font-size: 12px; padding: 8px; background: #fff8f8; border-left: 4px solid #ff4444; border-radius: 4px;">
        <h3 style="margin: 0 0 8px 0; color: #ff4444;">{props.get('name', 'Unknown')}</h3>
        <p style="margin: 4px 0;"><b>Site ID:</b> {props.get('site_id', 'N/A')}</p>
        <p style="margin: 4px 0;"><b>State:</b> {props.get('state', 'N/A')}</p>
        <p style="margin: 4px 0;"><b>Forest Type:</b> {forest_type}</p>
        <p style="margin: 4px 0;"><b>Max Height:</b> {props.get('expected_max_height_m', 'N/A')}m</p>
        <p style="margin: 4px 0;"><b>Priority:</b> {props.get('priority', 'N/A')}</p>
        <p style="margin: 4px 0;"><b>Description:</b> {props.get('description', 'N/A')}</p>
        <p style="margin: 4px 0; font-style: italic; color: #666;">{props.get('notes', '')}</p>
    </div>
    """

# Popup callback for lidar boundaries
def on_lidar_click(event, feature, **kwargs):
    props = feature['properties']
    name = props.get('name', 'Unknown')
    parsed = parse_lidar_name(name)
    count = props.get('count', 0)
    count_str = f"{count:,}" if count else "N/A"
    url = props.get('url', '')
    
    # Create clickable link if URL exists
    url_html = f'<a href="{url}" target="_blank" style="color: #0088cc; word-break: break-all;">EPT Endpoint</a>' if url else 'N/A'
    
    popup_html.value = f"""
    <div style="font-family: Arial, sans-serif; font-size: 12px; padding: 8px; background: #f0ffff; border-left: 4px solid #00cccc; border-radius: 4px;">
        <h3 style="margin: 0 0 8px 0; color: #00aaaa;">3DEP Lidar Dataset</h3>
        <p style="margin: 4px 0;"><b>Collection:</b> {name}</p>
        <p style="margin: 4px 0;"><b>State:</b> {parsed['state']}</p>
        <p style="margin: 4px 0;"><b>Location:</b> {parsed['location']}</p>
        <p style="margin: 4px 0;"><b>Year:</b> {parsed['year']}</p>
        <p style="margin: 4px 0;"><b>Point Count:</b> {count_str}</p>
        <p style="margin: 4px 0;"><b>Data:</b> {url_html}</p>
    </div>
    """

# Attach click handlers
sites_layer.on_click(on_site_click)
lidar_layer.on_click(on_lidar_click)

# Add controls
overview_map.add(FullScreenControl())
overview_map.add(ScaleControl(position='bottomleft'))
overview_map.add(LayersControl(position='topright'))

print("Overview Map - All Forest Sites with 3DEP Lidar Coverage")
print("Click on polygons to see details. Use layer control (top right) to toggle layers.")
from ipywidgets import VBox
VBox([overview_map, popup_html])

Overview Map - All Forest Sites with 3DEP Lidar Coverage
Click on polygons to see details. Use layer control (top right) to toggle layers.


## Site Summary Table

In [6]:
# Display site summary
print(f"{'Site ID':<30} {'Name':<35} {'State':<5} {'Priority':<8} {'Max Height':<10}")
print("-" * 95)

for site_id, site_data in sorted(sites.items(), key=lambda x: (x[1]['priority'], x[0])):
    print(f"{site_id:<30} {site_data['name']:<35} {site_data['state']:<5} {site_data['priority']:<8} {site_data['expected_max_height_m']:<10}m")

Site ID                        Name                                State Priority Max Height
-----------------------------------------------------------------------------------------------
great_smoky_cove               Great Smoky Mountains Cove Forest   TN    1        55        m
harvard_forest                 Harvard Forest LTER                 MA    1        30        m
redwood_humboldt               Humboldt Redwoods                   CA    1        115       m
sequoia_giant_forest           Sequoia Giant Forest                CA    1        95        m
wind_river                     Wind River Experimental Forest      WA    1        65        m
coconino_ponderosa             Coconino Ponderosa Pine             AZ    2        40        m
niwot_ridge                    Niwot Ridge LTER                    CO    2        20        m
ocala_longleaf                 Ocala Longleaf Pine                 FL    2        35        m
olympic_hoh                    Olympic Hoh Rainforest      

---

## Individual Site Maps

Below are detailed maps for each forest site, organized by region. Use these to examine forest structure and verify location accuracy.

In [9]:
from ipywidgets import VBox

def get_intersecting_lidar(site_bbox, lidar_geojson):
    """Get lidar features that intersect with site bbox."""
    west, south, east, north = site_bbox
    site_box = box(west, south, east, north)
    
    intersecting_features = []
    for feature in lidar_geojson['features']:
        try:
            lidar_geom = shape(feature['geometry'])
            if site_box.intersects(lidar_geom):
                intersecting_features.append(feature)
        except Exception:
            continue
    
    return {
        "type": "FeatureCollection",
        "features": intersecting_features
    }

def create_site_map(site_id, site_data, height='400px'):
    """
    Create an interactive map for a single forest site with lidar coverage.
    Includes click popups for site AOI and lidar boundaries.
    
    Args:
        site_id: The site identifier
        site_data: Dictionary with site configuration
        height: Map height (default 400px)
        
    Returns:
        ipywidgets.VBox containing map and popup HTML widget
    """
    bbox = site_data['bbox']  # [west, south, east, north]
    west, south, east, north = bbox
    
    # Calculate center
    center_lat = (south + north) / 2
    center_lon = (west + east) / 2
    
    # Create map
    m = Map(
        center=(center_lat, center_lon),
        zoom=13,
        layout=Layout(width='100%', height=height),
        scroll_wheel_zoom=True
    )
    
    # Add satellite basemap
    satellite = basemap_to_tiles(basemaps.Esri.WorldImagery)
    m.add(satellite)
    
    # Create HTML widget for popup display
    popup_html = HTML()
    popup_html.layout.width = '100%'
    
    # Get intersecting lidar boundaries for this site
    site_lidar = get_intersecting_lidar(bbox, lidar_boundaries)
    
    # Add lidar coverage layer (cyan)
    if site_lidar['features']:
        lidar_layer = GeoJSON(
            data=site_lidar,
            style={
                'color': '#00ffff',
                'weight': 2,
                'fillColor': '#00ffff',
                'fillOpacity': 0.1
            },
            hover_style={'fillOpacity': 0.25},
            name='3DEP Lidar'
        )
        m.add(lidar_layer)
        
        # Popup callback for lidar boundaries
        def on_lidar_click(event, feature, **kwargs):
            props = feature['properties']
            name = props.get('name', 'Unknown')
            parsed = parse_lidar_name(name)
            count = props.get('count', 0)
            count_str = f"{count:,}" if count else "N/A"
            url = props.get('url', '')
            
            # Create clickable link if URL exists
            url_html = f'<a href="{url}" target="_blank" style="color: #0088cc; word-break: break-all;">EPT Endpoint</a>' if url else 'N/A'
            
            popup_html.value = f"""
            <div style="font-family: Arial, sans-serif; font-size: 12px; padding: 8px; background: #f0ffff; border-left: 4px solid #00cccc; border-radius: 4px; margin-top: 8px;">
                <h4 style="margin: 0 0 6px 0; color: #00aaaa;">3DEP Lidar Dataset</h4>
                <p style="margin: 2px 0;"><b>Collection:</b> {name}</p>
                <p style="margin: 2px 0;"><b>State:</b> {parsed['state']}</p>
                <p style="margin: 2px 0;"><b>Location:</b> {parsed['location']}</p>
                <p style="margin: 2px 0;"><b>Year:</b> {parsed['year']}</p>
                <p style="margin: 2px 0;"><b>Point Count:</b> {count_str}</p>
                <p style="margin: 2px 0;"><b>Data:</b> {url_html}</p>
            </div>
            """
        lidar_layer.on_click(on_lidar_click)
    
    # Create polygon from bbox with full site metadata
    polygon_coords = [
        [west, south],
        [east, south],
        [east, north],
        [west, north],
        [west, south]
    ]
    
    geojson_data = {
        "type": "FeatureCollection",
        "features": [{
            "type": "Feature",
            "properties": {
                "site_id": site_id,
                "name": site_data['name'],
                "description": site_data['description'],
                "state": site_data.get('state', 'N/A'),
                "forest_type": site_data.get('forest_type', 'N/A'),
                "expected_max_height_m": site_data.get('expected_max_height_m', 'N/A'),
                "notes": site_data.get('notes', ''),
                "priority": site_data.get('priority', 'N/A')
            },
            "geometry": {
                "type": "Polygon",
                "coordinates": [polygon_coords]
            }
        }]
    }
    
    # Add site polygon layer (green)
    geo_layer = GeoJSON(
        data=geojson_data,
        style={
            'color': '#00ff00',
            'weight': 3,
            'fillColor': '#00ff00',
            'fillOpacity': 0.1
        },
        hover_style={'fillOpacity': 0.3},
        name='Site AOI'
    )
    m.add(geo_layer)
    
    # Popup callback for site AOI
    def on_site_click(event, feature, **kwargs):
        props = feature['properties']
        forest_type = props.get('forest_type', 'N/A')
        if forest_type != 'N/A':
            forest_type = forest_type.replace('_', ' ').title()
        popup_html.value = f"""
        <div style="font-family: Arial, sans-serif; font-size: 12px; padding: 8px; background: #f0fff0; border-left: 4px solid #228b22; border-radius: 4px; margin-top: 8px;">
            <h4 style="margin: 0 0 6px 0; color: #228b22;">{props.get('name', 'Unknown')}</h4>
            <p style="margin: 2px 0;"><b>Site ID:</b> {props.get('site_id', 'N/A')}</p>
            <p style="margin: 2px 0;"><b>State:</b> {props.get('state', 'N/A')}</p>
            <p style="margin: 2px 0;"><b>Forest Type:</b> {forest_type}</p>
            <p style="margin: 2px 0;"><b>Max Height:</b> {props.get('expected_max_height_m', 'N/A')}m</p>
            <p style="margin: 2px 0;"><b>Priority:</b> {props.get('priority', 'N/A')}</p>
            <p style="margin: 2px 0;"><b>Description:</b> {props.get('description', 'N/A')}</p>
            <p style="margin: 2px 0; font-style: italic; color: #666;">{props.get('notes', '')}</p>
        </div>
        """
    geo_layer.on_click(on_site_click)
    
    # Add controls
    m.add(FullScreenControl())
    m.add(ScaleControl(position='bottomleft'))
    m.add(LayersControl(position='topright'))
    
    return VBox([m, popup_html])

### Pacific Northwest - Temperate Rainforests

In [11]:
site_id = 'sequoia_giant_forest'
site = sites[site_id]
print(f"## {site['name']} ({site['state']})")
print(f"Description: {site['description']}")
print(f"Forest Type: {site['forest_type']}")
print(f"Expected Max Height: {site['expected_max_height_m']}m")
print(f"Notes: {site['notes']}")
print(f"Priority: {site['priority']}")
print(f"Bbox: {site['bbox']}")
create_site_map(site_id, site)

## Sequoia Giant Forest (CA)
Description: Home to the largest trees on Earth by volume
Forest Type: giant_sequoia
Expected Max Height: 95m
Notes: Contains General Sherman tree (84m tall, largest by volume)
Priority: 1
Bbox: [-118.77, 36.55, -118.72, 36.6]


In [12]:
site_id = 'redwood_humboldt'
site = sites[site_id]
print(f"## {site['name']} ({site['state']})")
print(f"Description: {site['description']}")
print(f"Forest Type: {site['forest_type']}")
print(f"Expected Max Height: {site['expected_max_height_m']}m")
print(f"Notes: {site['notes']}")
print(f"Priority: {site['priority']}")
print(f"Bbox: {site['bbox']}")
create_site_map(site_id, site)

## Humboldt Redwoods (CA)
Description: Old-growth coast redwood forest
Forest Type: coast_redwood
Expected Max Height: 115m
Notes: Tallest trees on Earth - Hyperion (115.92m) nearby
Priority: 1
Bbox: [-123.98, 40.3, -123.9, 40.38]


In [13]:
site_id = 'olympic_hoh'
site = sites[site_id]
print(f"## {site['name']} ({site['state']})")
print(f"Description: {site['description']}")
print(f"Forest Type: {site['forest_type']}")
print(f"Expected Max Height: {site['expected_max_height_m']}m")
print(f"Notes: {site['notes']}")
print(f"Priority: {site['priority']}")
print(f"Bbox: {site['bbox']}")
create_site_map(site_id, site)

## Olympic Hoh Rainforest (WA)
Description: Temperate rainforest with massive Sitka spruce
Forest Type: temperate_rainforest
Expected Max Height: 75m
Notes: One of few temperate rainforests in North America
Priority: 2
Bbox: [-123.95, 47.82, -123.88, 47.88]


In [14]:
site_id = 'tongass_mendenhall'
site = sites[site_id]
print(f"## {site['name']} ({site['state']})")
print(f"Description: {site['description']}")
print(f"Forest Type: {site['forest_type']}")
print(f"Expected Max Height: {site['expected_max_height_m']}m")
print(f"Notes: {site['notes']}")
print(f"Priority: {site['priority']}")
print(f"Bbox: {site['bbox']}")
create_site_map(site_id, site)

## Tongass Mendenhall Valley (AK)
Description: Largest US national forest
Forest Type: temperate_rainforest
Expected Max Height: 60m
Notes: Sitka spruce and western hemlock dominant
Priority: 2
Bbox: [-134.6, 58.38, -134.5, 58.45]


In [11]:
site_id = 'wind_river'
site = sites[site_id]
print(f"## {site['name']} ({site['state']})")
print(f"Description: {site['description']}")
print(f"Forest Type: {site['forest_type']}")
print(f"Expected Max Height: {site['expected_max_height_m']}m")
print(f"Notes: {site['notes']}")
print(f"Priority: {site['priority']}")
print(f"Bbox: {site['bbox']}")
create_site_map(site_id, site)

## Wind River Experimental Forest (WA)
Description: Old-growth Douglas-fir research site
Forest Type: douglas_fir
Expected Max Height: 65m
Notes: AmeriFlux tower site with carbon flux data
Priority: 1
Bbox: [-122.0, 45.8, -121.93, 45.86]


Map(center=[45.83, -121.965], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoo…

### Rocky Mountains - Conifer Forests

In [12]:
site_id = 'yellowstone_lodgepole'
site = sites[site_id]
print(f"## {site['name']} ({site['state']})")
print(f"Description: {site['description']}")
print(f"Forest Type: {site['forest_type']}")
print(f"Expected Max Height: {site['expected_max_height_m']}m")
print(f"Notes: {site['notes']}")
print(f"Priority: {site['priority']}")
print(f"Bbox: {site['bbox']}")
create_site_map(site_id, site)

## Yellowstone Lodgepole Pine (WY)
Description: Post-1988 fire regeneration study area
Forest Type: lodgepole_pine
Expected Max Height: 30m
Notes: Regenerating forest after 1988 fires
Priority: 2
Bbox: [-110.45, 44.55, -110.38, 44.62]


Map(center=[44.584999999999994, -110.41499999999999], controls=(ZoomControl(options=['position', 'zoom_in_text…

In [13]:
site_id = 'rocky_mountain_subalpine'
site = sites[site_id]
print(f"## {site['name']} ({site['state']})")
print(f"Description: {site['description']}")
print(f"Forest Type: {site['forest_type']}")
print(f"Expected Max Height: {site['expected_max_height_m']}m")
print(f"Notes: {site['notes']}")
print(f"Priority: {site['priority']}")
print(f"Bbox: {site['bbox']}")
create_site_map(site_id, site)

## Rocky Mountain Subalpine (CO)
Description: High-elevation spruce-fir forest
Forest Type: spruce_fir
Expected Max Height: 35m
Notes: Near treeline, Engelmann spruce and subalpine fir
Priority: 3
Bbox: [-105.65, 40.28, -105.58, 40.35]


Map(center=[40.315, -105.61500000000001], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_…

### Southwest - Ponderosa Pine

In [ ]:
site_id = 'coconino_ponderosa'
site = sites[site_id]
print(f"## {site['name']} ({site['state']})")
print(f"Description: {site['description']}")
print(f"Forest Type: {site['forest_type']}")
print(f"Expected Max Height: {site['expected_max_height_m']}m")
print(f"Notes: {site['notes']}")
print(f"Priority: {site['priority']}")
print(f"Bbox: {site['bbox']}")
create_site_map(site_id, site)

In [ ]:
site_id = 'gila_mixed_conifer'
site = sites[site_id]
print(f"## {site['name']} ({site['state']})")
print(f"Description: {site['description']}")
print(f"Forest Type: {site['forest_type']}")
print(f"Expected Max Height: {site['expected_max_height_m']}m")
print(f"Notes: {site['notes']}")
print(f"Priority: {site['priority']}")
print(f"Bbox: {site['bbox']}")
create_site_map(site_id, site)

### Eastern Deciduous - Hardwood Forests

In [ ]:
site_id = 'great_smoky_cove'
site = sites[site_id]
print(f"## {site['name']} ({site['state']})")
print(f"Description: {site['description']}")
print(f"Forest Type: {site['forest_type']}")
print(f"Expected Max Height: {site['expected_max_height_m']}m")
print(f"Notes: {site['notes']}")
print(f"Priority: {site['priority']}")
print(f"Bbox: {site['bbox']}")
create_site_map(site_id, site)

In [ ]:
site_id = 'mark_twain_ozark'
site = sites[site_id]
print(f"## {site['name']} ({site['state']})")
print(f"Description: {site['description']}")
print(f"Forest Type: {site['forest_type']}")
print(f"Expected Max Height: {site['expected_max_height_m']}m")
print(f"Notes: {site['notes']}")
print(f"Priority: {site['priority']}")
print(f"Bbox: {site['bbox']}")
create_site_map(site_id, site)

In [ ]:
site_id = 'white_mountain_northern'
site = sites[site_id]
print(f"## {site['name']} ({site['state']})")
print(f"Description: {site['description']}")
print(f"Forest Type: {site['forest_type']}")
print(f"Expected Max Height: {site['expected_max_height_m']}m")
print(f"Notes: {site['notes']}")
print(f"Priority: {site['priority']}")
print(f"Bbox: {site['bbox']}")
create_site_map(site_id, site)

### Boreal/Northern - Mixed Forests

In [ ]:
site_id = 'boundary_waters'
site = sites[site_id]
print(f"## {site['name']} ({site['state']})")
print(f"Description: {site['description']}")
print(f"Forest Type: {site['forest_type']}")
print(f"Expected Max Height: {site['expected_max_height_m']}m")
print(f"Notes: {site['notes']}")
print(f"Priority: {site['priority']}")
print(f"Bbox: {site['bbox']}")
create_site_map(site_id, site)

In [ ]:
site_id = 'superior_old_growth'
site = sites[site_id]
print(f"## {site['name']} ({site['state']})")
print(f"Description: {site['description']}")
print(f"Forest Type: {site['forest_type']}")
print(f"Expected Max Height: {site['expected_max_height_m']}m")
print(f"Notes: {site['notes']}")
print(f"Priority: {site['priority']}")
print(f"Bbox: {site['bbox']}")
create_site_map(site_id, site)

### Southeastern - Pine Forests

In [ ]:
site_id = 'ocala_longleaf'
site = sites[site_id]
print(f"## {site['name']} ({site['state']})")
print(f"Description: {site['description']}")
print(f"Forest Type: {site['forest_type']}")
print(f"Expected Max Height: {site['expected_max_height_m']}m")
print(f"Notes: {site['notes']}")
print(f"Priority: {site['priority']}")
print(f"Bbox: {site['bbox']}")
create_site_map(site_id, site)

In [ ]:
site_id = 'conecuh_longleaf'
site = sites[site_id]
print(f"## {site['name']} ({site['state']})")
print(f"Description: {site['description']}")
print(f"Forest Type: {site['forest_type']}")
print(f"Expected Max Height: {site['expected_max_height_m']}m")
print(f"Notes: {site['notes']}")
print(f"Priority: {site['priority']}")
print(f"Bbox: {site['bbox']}")
create_site_map(site_id, site)

### Special Research Sites

In [ ]:
site_id = 'harvard_forest'
site = sites[site_id]
print(f"## {site['name']} ({site['state']})")
print(f"Description: {site['description']}")
print(f"Forest Type: {site['forest_type']}")
print(f"Expected Max Height: {site['expected_max_height_m']}m")
print(f"Notes: {site['notes']}")
print(f"Priority: {site['priority']}")
print(f"Bbox: {site['bbox']}")
create_site_map(site_id, site)

In [ ]:
site_id = 'niwot_ridge'
site = sites[site_id]
print(f"## {site['name']} ({site['state']})")
print(f"Description: {site['description']}")
print(f"Forest Type: {site['forest_type']}")
print(f"Expected Max Height: {site['expected_max_height_m']}m")
print(f"Notes: {site['notes']}")
print(f"Priority: {site['priority']}")
print(f"Bbox: {site['bbox']}")
create_site_map(site_id, site)

---

## Notes on Site Verification

When examining each site, look for:

1. **Lidar Coverage**: Does the cyan overlay show 3DEP lidar data is available?
2. **Forest Coverage**: Is the AOI actually covered by forest?
3. **Forest Type**: Does the visible canopy match the expected forest type?
4. **Access**: Are there roads or trails visible that might affect the analysis?
5. **Disturbance**: Are there signs of logging, fire, or other disturbances?
6. **Boundary Accuracy**: Does the bounding box capture the intended forest area?

## Map Legend

- **Green polygon**: Site Area of Interest (AOI)
- **Cyan polygon**: 3DEP lidar flight boundary (indicates available data)

## GeoJSON Files

Individual GeoJSON files for each site are available in the `../geojson/` directory:
- `all_sites.geojson` - Combined file with all sites
- `<site_id>.geojson` - Individual site files

## 3DEP Lidar Data Source

Lidar boundaries from: https://github.com/hobuinc/usgs-lidar

In [ ]:
# List generated GeoJSON files
import os

geojson_files = sorted(geojson_dir.glob('*.geojson'))
print(f"Generated GeoJSON files ({len(geojson_files)} total):")
for f in geojson_files:
    size_kb = f.stat().st_size / 1024
    print(f"  {f.name:<40} ({size_kb:.1f} KB)")